# Introduction

Any individual wishing to start a restaurant wants nothing more than for his/her restaurant to succeed. If location is not an issue, one option is to figure out which neighborhood(s) will most enjoy the cuisine that the individual wishes to serve. On the other hand, if the restaurant owner is set on establishing themselves in a particular neighborhood, then it may be advisable to investigate said neighborhood's tastes and preferences regarding its cuisine. Thus, establishing which neighborhoods like which cuisines is of paramount importance to any restaurant owner.

# Data

I will be focusing on the center of the city of London for this project. I will be using foursquare's data on venue locations to determine which cuisines are most popular in the center of London. I will determine this preferance by the frequency of each cuisine type in the immediate area. The website from which the postal code data was pulled is https://www.doogal.co.uk/UKPostcodes.php?Search=W1J. The information relating to the venues was extracted via the four square API.   

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

import json # library to handle JSON files

!conda install pip

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!pip install folium
import folium # map rendering library


print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

     |████████████████████████████████| 93 kB 3.7 MB/s  eta 0:00:01
Libraries imported.


In [2]:
# Read in Dataframe
df1 = pd.read_csv("https://www.doogal.co.uk/UKPostcodesCSV.ashx?Search=W1J")
df1.head()

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,...,Nearest station,Distance to station,Postcode area,Postcode district,Police force,Water company,Plus Code,Average Income,Sewage Company,Travel To Work Area
0,W1J 0AA,Yes,51.509092,-0.137555,529347,180551,TQ293805,Greater London,Westminster,West End,...,Piccadilly Circus,0.274171,W,W1J,Metropolitan Police,Thames Water,9C3XGV56+JX,54000,NaN,London
1,W1J 0AB,Yes,51.509537,-0.136125,529445,180603,TQ294806,Greater London,Westminster,West End,...,Piccadilly Circus,0.167850,W,W1J,Metropolitan Police,Thames Water,9C3XGV57+RH,54000,NaN,London
2,W1J 0AD,Yes,51.509657,-0.136181,529441,180616,TQ294806,Greater London,Westminster,West End,...,Piccadilly Circus,0.170839,W,W1J,Metropolitan Police,Thames Water,9C3XGV57+VG,54000,NaN,London
3,W1J 0AE,No,51.509745,-0.135641,529478,180627,TQ294806,Greater London,Westminster,West End,...,Piccadilly Circus,0.133543,W,W1J,Metropolitan Police,NaN,9C3XGV57+VP,54000,NaN,London
4,W1J 0AG,No,51.509647,-0.137403,529356,180613,TQ293806,Greater London,Westminster,West End,...,Piccadilly Circus,0.255354,W,W1J,Metropolitan Police,NaN,9C3XGV57+V2,54000,NaN,London


In [3]:
df1.columns

Index(['Postcode', 'In Use?', 'Latitude', 'Longitude', 'Easting', 'Northing',
       'Grid Ref', 'County', 'District', 'Ward', 'District Code', 'Ward Code',
       'Country', 'County Code', 'Constituency', 'Introduced', 'Terminated',
       'Parish', 'National Park', 'Population', 'Households', 'Built up area',
       'Built up sub-division', 'Lower layer super output area', 'Rural/urban',
       'Region', 'Altitude', 'London zone', 'LSOA Code', 'Local authority',
       'MSOA Code', 'Middle layer super output area', 'Parish Code',
       'Census output area', 'Constituency Code',
       'Index of Multiple Deprivation', 'Quality', 'User Type', 'Last updated',
       'Nearest station', 'Distance to station', 'Postcode area',
       'Postcode district', 'Police force', 'Water company', 'Plus Code',
       'Average Income', 'Sewage Company', 'Travel To Work Area'],
      dtype='object')

In [4]:
df1['London zone']

0      1
1      1
2      1
3      1
4      1
      ..
935    1
936    1
937    1
938    1
939    1
Name: London zone, Length: 940, dtype: int64

In [5]:
df = df1[['Postcode', 'In Use?', 'Grid Ref', 'Latitude', 'Longitude', 'District', 'Ward', 'Nearest station', 'Average Income']]
df.head()

,Postcode,In Use?,Grid Ref,Latitude,Longitude,District,Ward,Nearest station,Average Income
0,W1J 0AA,Yes,TQ293805,51.509092,-0.137555,Westminster,West End,Piccadilly Circus,54000
1,W1J 0AB,Yes,TQ294806,51.509537,-0.136125,Westminster,West End,Piccadilly Circus,54000
2,W1J 0AD,Yes,TQ294806,51.509657,-0.136181,Westminster,West End,Piccadilly Circus,54000
3,W1J 0AE,No,TQ294806,51.509745,-0.135641,Westminster,West End,Piccadilly Circus,54000
4,W1J 0AG,No,TQ293806,51.509647,-0.137403,Westminster,West End,Piccadilly Circus,54000


In [6]:
df.shape

(940, 9)

In [7]:
df.describe()

,Latitude,Longitude,Average Income
count,940.000000,940.000000,940.000000
mean,51.508008,-0.144914,58168.723404
std,0.001833,0.003950,2661.707410
min,51.502586,-0.152688,53800.000000
25%,51.506579,-0.147539,59000.000000
50%,51.508085,-0.145558,59000.000000
75%,51.509258,-0.143563,59000.000000
max,51.524565,-0.112042,73500.000000


In [8]:
df.columns = ['Postcode','In_Use','Grid_ref', 'Latitude', 'Longitude', 'District', 'Ward','Nearest_station','Average_Income']
df.head(10)

,Postcode,In_Use,Grid_ref,Latitude,Longitude,District,Ward,Nearest_station,Average_Income
0,W1J 0AA,Yes,TQ293805,51.509092,-0.137555,Westminster,West End,Piccadilly Circus,54000
1,W1J 0AB,Yes,TQ294806,51.509537,-0.136125,Westminster,West End,Piccadilly Circus,54000
2,W1J 0AD,Yes,TQ294806,51.509657,-0.136181,Westminster,West End,Piccadilly Circus,54000
3,W1J 0AE,No,TQ294806,51.509745,-0.135641,Westminster,West End,Piccadilly Circus,54000
4,W1J 0AG,No,TQ293806,51.509647,-0.137403,Westminster,West End,Piccadilly Circus,54000
5,W1J 0AH,Yes,TQ293806,51.509567,-0.137449,Westminster,West End,Piccadilly Circus,54000
6,W1J 0AJ,Yes,TQ291804,51.507825,-0.141008,Westminster,West End,Green Park,59000
7,W1J 0AL,Yes,TQ292805,51.509101,-0.138679,Westminster,West End,Piccadilly Circus,54000
8,W1J 0AN,Yes,TQ292805,51.509333,-0.139116,Westminster,West End,Green Park,54000
9,W1J 0AP,Yes,TQ292806,51.509671,-0.139463,Westminster,West End,Green Park,54000


In [9]:
# Remove Unused postal codes
df2 = df[df.In_Use!='No']
df2.head()

,Postcode,In_Use,Grid_ref,Latitude,Longitude,District,Ward,Nearest_station,Average_Income
0,W1J 0AA,Yes,TQ293805,51.509092,-0.137555,Westminster,West End,Piccadilly Circus,54000
1,W1J 0AB,Yes,TQ294806,51.509537,-0.136125,Westminster,West End,Piccadilly Circus,54000
2,W1J 0AD,Yes,TQ294806,51.509657,-0.136181,Westminster,West End,Piccadilly Circus,54000
5,W1J 0AH,Yes,TQ293806,51.509567,-0.137449,Westminster,West End,Piccadilly Circus,54000
6,W1J 0AJ,Yes,TQ291804,51.507825,-0.141008,Westminster,West End,Green Park,59000


In [10]:
df2.describe()

,Latitude,Longitude,Average_Income
count,695.000000,695.000000,695.000000
mean,51.507898,-0.145025,58132.086331
std,0.001794,0.003854,2457.129389
min,51.503410,-0.151864,53800.000000
25%,51.506495,-0.147590,59000.000000
50%,51.507932,-0.145824,59000.000000
75%,51.509118,-0.143569,59000.000000
max,51.524565,-0.112042,73500.000000


In [11]:
df2.shape

(695, 9)

In [12]:
CLIENT_ID = '1LO0V1TKCMUMTJCATD1JZOVSAWAF4OGIUZW0AUC15CJ54ZUR' # your Foursquare ID
CLIENT_SECRET = 'C24N0KQV5J3LDYN21DBFEAGZG31ZNT4ECMHEBNJXAH30OZTY' # your Foursquare Secret
VERSION = '20180604'

In [13]:
latitude = 51.509567
longitude = -0.137449
search_query = 'Restaurant'
radius = 1000
LIMIT = 300

In [14]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=1LO0V1TKCMUMTJCATD1JZOVSAWAF4OGIUZW0AUC15CJ54ZUR&client_secret=C24N0KQV5J3LDYN21DBFEAGZG31ZNT4ECMHEBNJXAH30OZTY&ll=51.509567,-0.137449&v=20180604&query=Restaurant&radius=1000&limit=300'

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fc930c0e208c61b1688f0e9'},
 'response': {'venues': [{'id': '5001b8e6e4b05e79b0b3b84d',
    'name': "Bill's Restaurant",
    'location': {'address': '36-44 Brewer St',
     'crossStreet': 'Lexington St',
     'lat': 51.51183377093123,
     'lng': -0.13508045829141016,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.51183377093123,
       'lng': -0.13508045829141016}],
     'distance': 300,
     'postalCode': 'W1F 9TB',
     'cc': 'GB',
     'city': 'London',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ['36-44 Brewer St (Lexington St)',
      'Soho',
      'Greater London',
      'W1F 9TB',
      'United Kingdom']},
    'categories': [{'id': '52e81612bcbc57f1066b7a05',
      'name': 'English Restaurant',
      'pluralName': 'English Restaurants',
      'shortName': 'English',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'pr

In [16]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,5001b8e6e4b05e79b0b3b84d,Bill's Restaurant,"[{'id': '52e81612bcbc57f1066b7a05', 'name': 'E...",v-1607020736,False,36-44 Brewer St,Lexington St,51.511834,-0.135080,"[{'label': 'display', 'lat': 51.51183377093123...",300,W1F 9TB,GB,London,Greater London,United Kingdom,"[36-44 Brewer St (Lexington St), Soho, Greater...",NaN,NaN
1,4afc9a04f964a5204c2422e3,C&R Café Restaurant,"[{'id': '4bf58dd8d48988d156941735', 'name': 'M...",v-1607020736,False,4-5 Rupert Court,NaN,51.511111,-0.132445,"[{'label': 'display', 'lat': 51.51111126738831...",386,W1D 6DY,GB,Chinatown,Greater London,United Kingdom,"[4-5 Rupert Court, Chinatown, Greater London, ...",Chinatown,NaN
2,4ac518caf964a520bda520e3,The Elysee Restaurant and Roof Garden,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",v-1607020736,False,13 Percy St,NaN,51.518270,-0.132954,"[{'label': 'display', 'lat': 51.51827036363987...",1017,W1T 1DP,GB,London,Greater London,United Kingdom,"[13 Percy St, London, Greater London, W1T 1DP,...",NaN,NaN
3,4b5344aef964a5200c9527e3,The Ivy Restaurant,"[{'id': '52e81612bcbc57f1066b7a05', 'name': 'E...",v-1607020736,False,1-5 West St,Litchfield St,51.512800,-0.127981,"[{'label': 'display', 'lat': 51.51279973860704...",748,WC2H 9NQ,GB,London,Greater London,United Kingdom,"[1-5 West St (Litchfield St), London, Greater ...",NaN,NaN
4,4c39bcafedba9521cf6cd625,New Loon Fung Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1607020736,False,42-44 Gerrard St,NaN,51.512039,-0.130371,"[{'label': 'display', 'lat': 51.51203891754673...",562,W1D 5QG,GB,London,Greater London,United Kingdom,"[42-44 Gerrard St, London, Greater London, W1D...",NaN,NaN


In [17]:
dataframe.shape

(50, 19)

In [18]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Bill's Restaurant,English Restaurant,36-44 Brewer St,Lexington St,51.511834,-0.135080,"[{'label': 'display', 'lat': 51.51183377093123...",300,W1F 9TB,GB,London,Greater London,United Kingdom,"[36-44 Brewer St (Lexington St), Soho, Greater...",NaN,5001b8e6e4b05e79b0b3b84d
1,C&R Café Restaurant,Malay Restaurant,4-5 Rupert Court,NaN,51.511111,-0.132445,"[{'label': 'display', 'lat': 51.51111126738831...",386,W1D 6DY,GB,Chinatown,Greater London,United Kingdom,"[4-5 Rupert Court, Chinatown, Greater London, ...",Chinatown,4afc9a04f964a5204c2422e3
2,The Elysee Restaurant and Roof Garden,Greek Restaurant,13 Percy St,NaN,51.518270,-0.132954,"[{'label': 'display', 'lat': 51.51827036363987...",1017,W1T 1DP,GB,London,Greater London,United Kingdom,"[13 Percy St, London, Greater London, W1T 1DP,...",NaN,4ac518caf964a520bda520e3
3,The Ivy Restaurant,English Restaurant,1-5 West St,Litchfield St,51.512800,-0.127981,"[{'label': 'display', 'lat': 51.51279973860704...",748,WC2H 9NQ,GB,London,Greater London,United Kingdom,"[1-5 West St (Litchfield St), London, Greater ...",NaN,4b5344aef964a5200c9527e3
4,New Loon Fung Chinese Restaurant,Chinese Restaurant,42-44 Gerrard St,NaN,51.512039,-0.130371,"[{'label': 'display', 'lat': 51.51203891754673...",562,W1D 5QG,GB,London,Greater London,United Kingdom,"[42-44 Gerrard St, London, Greater London, W1D...",NaN,4c39bcafedba9521cf6cd625
5,London Chinatown Restaurant,Chinese Restaurant,27 Gerrard St,NaN,51.511507,-0.131810,"[{'label': 'display', 'lat': 51.51150741274579...",446,W1D 6JN,GB,London,Greater London,United Kingdom,"[27 Gerrard St, London, Greater London, W1D 6J...",NaN,4ed23e3b2c5bc35059276f87
6,Bill's Restaurant,English Restaurant,28 St Martin's Courtyard,Upper St Martin's Ln,51.512555,-0.126600,"[{'label': 'display', 'lat': 51.51255465283461...",821,WC2E 9AB,GB,Covent Garden,Greater London,United Kingdom,[28 St Martin's Courtyard (Upper St Martin's L...,NaN,4d440bf1c3e5f04d0a219620
7,Al Hamra Restaurant,Lebanese Restaurant,31-33 Shepherd Mkt,NaN,51.506434,-0.147142,"[{'label': 'display', 'lat': 51.50643435933546...",756,W1J 7PT,GB,London,Greater London,United Kingdom,"[31-33 Shepherd Mkt, London, Greater London, W...",NaN,4beb16d2183895212eed0acf
8,Kowloon Restaurant and Cakeshop,Chinese Restaurant,21-22 Gerrard St,NaN,51.511503,-0.131904,"[{'label': 'display', 'lat': 51.51150273700102...",440,W1D 6JH,GB,Chinatown,Greater London,United Kingdom,"[21-22 Gerrard St, Chinatown, Greater London, ...",NaN,4b483c31f964a520754a26e3
9,Josephine's Restaurant,Filipino Restaurant,4 Charlotte St,NaN,51.518231,-0.134451,"[{'label': 'display', 'lat': 51.51823074429880...",986,W1T 2LP,GB,London,Greater London,United Kingdom,"[4 Charlotte St, London, Greater London, W1T 2...",NaN,4e48286f14959d51c17a21de


In [19]:
# Get the number of each type of restaurant within the predetermined radius
dataframe_filtered.groupby('categories').count()

,name,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
categories,,,,,,,,,,,,,,,
Asian Restaurant,2,2,0,2,2,2,2,2,2,2,2,2,2,1,2
Bar,3,3,0,3,3,3,3,3,3,3,3,3,3,0,3
Café,2,2,1,2,2,2,2,2,2,2,2,2,2,0,2
Cantonese Restaurant,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1
Chinese Restaurant,7,4,0,7,7,7,7,7,7,7,7,7,7,0,7
English Restaurant,4,4,4,4,4,4,4,4,4,4,4,4,4,0,4
Filipino Restaurant,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1
French Restaurant,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1
Greek Restaurant,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1


In [20]:
# Determine which restaurants are listed generically
df_1 = dataframe_filtered[dataframe_filtered['categories'] == 'Restaurant']
df_1.head(6)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
10,Restaurant on 5,Restaurant,334-348 Oxford St,NaN,51.515030,-0.148048,"[{'label': 'display', 'lat': 51.51503040765356...",953,NaN,GB,London,Greater London,United Kingdom,"[334-348 Oxford St, London, Greater London, Un...",NaN,526bd98211d29b555043e2c9
14,Madison's Bar Restaurant,Restaurant,The Washington Mayfair Hotel,NaN,51.507312,-0.150953,"[{'label': 'display', 'lat': 51.50731243306735...",968,W1J 5HE,GB,Mayfair,Greater London,United Kingdom,"[The Washington Mayfair Hotel, Mayfair, Greate...",NaN,51327638e4b041d1d2272263
19,The restaurant,Restaurant,Park Place,St James's Street,51.505948,-0.141607,"[{'label': 'display', 'lat': 51.50594758618708...",495,NaN,GB,London,Greater London,United Kingdom,"[Park Place (St James's Street), London, Great...",NaN,4feb3ecce4b0a4b22abef8c6
34,King Street Garden Restaurant,Restaurant,NaN,NaN,51.507157,-0.134562,"[{'label': 'display', 'lat': 51.50715688629132...",334,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,53c04e38498e72c336cb0941
37,Imli Restaurant,Restaurant,231 The Linen Hall,NaN,51.512722,-0.140419,"[{'label': 'display', 'lat': 51.51272212041476...",407,W1B 5TB,GB,162 Regent Street,NaN,United Kingdom,"[231 The Linen Hall, 162 Regent Street, W1B 5T...",NaN,5048d17470432e263db001c3
40,Rooftop Bar & Restaurant,Restaurant,Leicester Square 1,NaN,51.510765,-0.130666,"[{'label': 'display', 'lat': 51.510765, 'lng':...",488,NaN,GB,London,Greater London,United Kingdom,"[Leicester Square 1, London, Greater London, U...",NaN,5ad305fbeb449939d82aa665


In [21]:
df3 = dataframe_filtered[dataframe_filtered['categories'] == 'Asian Restaurant']
df4 = dataframe_filtered[dataframe_filtered['categories'] == 'Bar']
df5 = dataframe_filtered[dataframe_filtered['categories'] == 'Café']
df6 = dataframe_filtered[dataframe_filtered['categories'] == 'Cantonese Restaurant']
df7 = dataframe_filtered[dataframe_filtered['categories'] == 'Chinese Restaurant']
df8 = dataframe_filtered[dataframe_filtered['categories'] == 'English Restaurant']
df9 = dataframe_filtered[dataframe_filtered['categories'] == 'Filipino Restaurant']
df10 = dataframe_filtered[dataframe_filtered['categories'] == 'French Restaurant']
df11 = dataframe_filtered[dataframe_filtered['categories'] == 'Greek Restaurant']
df12 = dataframe_filtered[dataframe_filtered['categories'] == 'Hotel Bar']
df13 = dataframe_filtered[dataframe_filtered['categories'] == 'Indian Restaurant']
df14 = dataframe_filtered[dataframe_filtered['categories'] == 'Italian Restaurant']
df15 = dataframe_filtered[dataframe_filtered['categories'] == 'Japenese Restaurant']
df16 = dataframe_filtered[dataframe_filtered['categories'] == 'Korean Restaurant']
df17 = dataframe_filtered[dataframe_filtered['categories'] == 'Lebanese Restaurant']
df18 = dataframe_filtered[dataframe_filtered['categories'] == 'Malay Restaurant']
df19 = dataframe_filtered[dataframe_filtered['categories'] == 'Persian Restaurant']
df20 = dataframe_filtered[dataframe_filtered['categories'] == 'Swiss Restaurant']
df21 = dataframe_filtered[dataframe_filtered['categories'] == 'Szechuan Restaurant']
df22 = dataframe_filtered[dataframe_filtered['categories'] == 'Turkish Restaurant']
df23 = dataframe_filtered[dataframe_filtered['categories'] == 'Vegetarian / Vegan Restaurant']
df24 = dataframe_filtered[dataframe_filtered['categories'] == 'Wine Bar']

In [23]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=14) # generate map centred around Westminster

# add a red circle marker to represent Westminster
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Asian restaurants as blue circle markers
for lat, lng, label in zip(df3.lat, df3.lng, df3.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# add the Bars as yellow circle markers
for lat, lng, label in zip(df4.lat, df4.lng, df4.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.8
    ).add_to(venues_map)    

# add the Cafes as green circle markers
for lat, lng, label in zip(df5.lat, df5.lng, df5.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.8
    ).add_to(venues_map) 
    
# add the Cantonese Restaurants as purple circle markers
for lat, lng, label in zip(df6.lat, df6.lng, df6.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=label,
        fill = True,
        fill_color='purple',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Chinese Restaurants as orange circle markers
for lat, lng, label in zip(df7.lat, df7.lng, df7.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.8
    ).add_to(venues_map)

# add the English Restaurants as pink circle markers
for lat, lng, label in zip(df8.lat, df8.lng, df8.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='pink',
        popup=label,
        fill = True,
        fill_color='pink',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Filipino Restaurants as black circle markers
for lat, lng, label in zip(df9.lat, df9.lng, df9.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='black',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the French Restaurants as dark red circle markers
for lat, lng, label in zip(df10.lat, df10.lng, df10.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='darkred',
        popup=label,
        fill = True,
        fill_color='darkred',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Greek Restaurants as dark green circle markers
for lat, lng, label in zip(df11.lat, df11.lng, df11.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='darkgreen',
        popup=label,
        fill = True,
        fill_color='darkgreen',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Hotel Bars as dark purple circle markers
for lat, lng, label in zip(df12.lat, df12.lng, df12.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='darkpurple',
        popup=label,
        fill = True,
        fill_color='darkpurple',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Indian Restaurants as dark blue circle markers
for lat, lng, label in zip(df13.lat, df13.lng, df13.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='darkblue',
        popup=label,
        fill = True,
        fill_color='darkblue',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Italian Restaurants as cadet blue circle markers
for lat, lng, label in zip(df14.lat, df14.lng, df14.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='cadetblue',
        popup=label,
        fill = True,
        fill_color='cadetblue',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Japenese Restaurants as light blue circle markers
for lat, lng, label in zip(df15.lat, df15.lng, df15.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='lightblue',
        popup=label,
        fill = True,
        fill_color='lightblue',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Korean Restaurants as light green circle markers
for lat, lng, label in zip(df16.lat, df16.lng, df16.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='lightgreen',
        popup=label,
        fill = True,
        fill_color='lightgreen',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Lebanese Restaurants as light red circle markers
for lat, lng, label in zip(df17.lat, df17.lng, df17.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='lightred',
        popup=label,
        fill = True,
        fill_color='lightred',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Malay Restaurants as black/blue circle markers
for lat, lng, label in zip(df18.lat, df18.lng, df18.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Persian Restaurants as black/green circle markers
for lat, lng, label in zip(df19.lat, df19.lng, df19.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Swiss Restaurants as black/red circle markers
for lat, lng, label in zip(df20.lat, df20.lng, df20.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Szechuan Restaurants as black/pink circle markers
for lat, lng, label in zip(df21.lat, df21.lng, df21.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='pink',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Turkish Restaurants as black/white circle markers
for lat, lng, label in zip(df22.lat, df22.lng, df22.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='white',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Vegetarian / Vegan Restaurants as blue/red circle markers
for lat, lng, label in zip(df23.lat, df23.lng, df23.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add the Wine Bars as blue/yellow circle markers
for lat, lng, label in zip(df9.lat, df9.lng, df9.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# display map
venues_map